Assignment Objective: You are required to design, evaluate, and critically interpret K-Nearest Neighbor(KNN)models on these datasets representing classification and regression tasks. This exercise is intended to build your competency in: 
• Exploratory Data Analysis (EDA) and preprocessing in applied contexts
• Model design decisions informed by empirical evidence
• Strengths and limitations of KNN as a non-parametric algorithm
• Effective communication of machine learning results and domain insights

Question 1:
Download the following dataset (from Kaggle) with 110.527 medical appointments and 14 associated variables (characteristics). Use it to create models that can predict whether a patient will show up for the appointment booked or not.
https://www.kaggle.com/datasets/joniarroba/noshowappointments

Step 1: Importing Libraries and Preprocessing the Dataset

In [1]:
# Importing libraries
import numpy as np
import pandas as pd
import kagglehub
import matplotlib.pyplot as plt

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Loading the dataset
path = kagglehub.dataset_download("joniarroba/noshowappointments")
print("Path to dataset files:", path)
df = pd.read_csv(f"{path}/KaggleV2-May-2016.csv")    # Load the CSV file
df.head() # Display the first few rows

/home/linuxbrew/.linuxbrew/Cellar/python@3.11/3.11.4_1/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
